In [1]:
# -*- coding: utf-8
# The Intercept Brasil (https://theintercept.com/brasil/)
# Reinaldo Chaves (@paidatocandeira)
# Verifica se empresas encontradas tiveram ordem bancárias recentes com o governo do RJ
#

In [58]:
import pandas as pd
import unidecode
from pathlib import Path

Função para retirar acentuação de nomes

In [59]:
def f(str):
    return (unidecode.unidecode(str))

Função para ler a pasta com as Ordens Bancárias

In [60]:
def _reader_for(fname):
    return pd.read_csv(fname, sep=';', header=0, encoding='latin_1', decimal = ",", converters={'Favorecido': lambda x: str(x)})

Função para checar se todos os CNPJ têm 14 dígitos

In [61]:
def compara(linha):
    if len(linha['cnpj']) < 14:
        conta = 14 - (len(linha['cnpj']))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + linha['cnpj']
    else:
        alterado = linha['cnpj']
    return alterado

Dados de Ordens Bancárias obtidos em http://www.transparencia.rj.gov.br/transparencia/faces/OrcamentoTematico/fornecedor

Entre janeiro de 2017 e abril de 2018

In [62]:
folder = Path("fornecedores_rj")
fornecedores_rj = pd.concat([
    _reader_for(csv)
    for csv in folder.glob("*.csv")
])

In [63]:
fornecedores_rj.rename(columns={'Valor OB':'Valor_OB'}, inplace=True)

In [64]:
fornecedores_rj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139036 entries, 0 to 9591
Data columns (total 11 columns):
Data do Pagamento    139036 non-null object
Favorecido           139036 non-null object
Nome Favorecido      139036 non-null object
UG                   139036 non-null int64
Título UG            139036 non-null object
Código Órgão         139036 non-null int64
Órgão                139036 non-null object
OB                   139036 non-null object
Histórico            139036 non-null object
Valor_OB             139036 non-null object
Unnamed: 10          0 non-null float64
dtypes: float64(1), int64(2), object(8)
memory usage: 12.7+ MB


In [65]:
fornecedores_rj.reset_index().head()

,index,Data do Pagamento,Favorecido,Nome Favorecido,UG,Título UG,Código Órgão,Órgão,OB,Histórico,Valor_OB,Unnamed: 10
0,0,01/02/2018,02150336000166,Aguas de Niteroi S/A,36100,FUNDO ESPECIAL DO TRIBUNAL DE JUSTICA,3,FUNDO ESPECIAL DO TRIBUNAL DE JUSTICA,2018OB01389,LIQUIDAÇÃO DO (S) DOCUMENTO (S) 700019707 CO...,"24.756,57",nan
1,1,01/02/2018,11286829000119,"BH FILTROS, BEBEDOUROS E PURIF DE AGUA LTDA-ME",20100,TRIBUNAL DE CONTAS,2,Tribunal de Contas do Estado do Rio de Janeiro,2018OB00090,PAGAMENTO DO (S) DOCUMENTO (S) N.F.4154/17 C...,"1.438,80",nan
2,2,01/02/2018,29884632000147,Infotec Consultoria E Planejamento Ltda.,36100,FUNDO ESPECIAL DO TRIBUNAL DE JUSTICA,3,FUNDO ESPECIAL DO TRIBUNAL DE JUSTICA,2018OB01380,LIQUIDAÇÃO DO (S) DOCUMENTO (S) 5314 CONFORM...,"163.856,00",nan
3,3,01/02/2018,14470588000151,SERVIOESTE RIO DE JANEIRO LTDA,266500,FUNDO DA POLICIA MILITAR DO ERJ,26,FUNDO ESPECIAL DA POLICIA MILITAR DO ERJ,2018OB00335,LIQUIDAÇÃO DO (S) DOCUMENTO (S) 05 CONFORME ...,"29,19",nan
4,4,01/02/2018,72734791000194,Quimiclean Comercio E Servicos Ltda-epp,20100,TRIBUNAL DE CONTAS,2,Tribunal de Contas do Estado do Rio de Janeiro,2018OB00108,PAGAMENTO DO (S) DOCUMENTO (S) N.F.5696/18 C...,"830,92",nan


O arquivo gerado em etapas anteriores foi checado, limpo e preenchido manualmente. Carrega o resultado. São empresas em geral de segurança privada

In [84]:
consolidado = pd.read_csv("empresas_de_seguranca_consolidado_final_nomes.csv",sep=',',encoding = 'utf-8', converters={'cnpj': lambda x: str(x)})

Filtra apenas empresas de seguranca privada que tenham sócios ou administradores vindo de forças de segurança

In [96]:
policiais = consolidado[(consolidado['cargo_relacionado_socio'] == 'Administracao Penitenciaria') | 
                        (consolidado['cargo_relacionado_socio'] == 'Aeronautica') | 
                        (consolidado['cargo_relacionado_socio'] == 'Agente penitenciario') |  
                        (consolidado['cargo_relacionado_socio'] == 'Bombeiros') | 
                        (consolidado['cargo_relacionado_socio'] == 'Empresario - ligado a policial civil') | 
                        (consolidado['cargo_relacionado_socio'] == 'Exercito') | 
                        (consolidado['cargo_relacionado_socio'] == 'Marinha') | 
                        (consolidado['cargo_relacionado_socio'] == 'Policia Civil') |  
                        (consolidado['cargo_relacionado_socio'] == 'Policia Federal') | 
                        (consolidado['cargo_relacionado_socio'] == 'Policia Militar') | 
                        (consolidado['cargo_relacionado_socio'] == 'Secretario de Seguranca')]

In [97]:
policiais.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 59 to 1222
Data columns (total 30 columns):
cnpj                                187 non-null object
CNAE_principal                      177 non-null object
nome_empresa                        187 non-null object
nome_empresa_maiuscula_minuscula    157 non-null object
estado                              157 non-null object
indicador                           178 non-null float64
documento                           178 non-null object
qualificacao                        178 non-null object
socio                               181 non-null object
socio_maiuscula_minuscula           160 non-null object
data_abertura                       187 non-null object
logradouro                          187 non-null object
numero_logradouro                   187 non-null object
complemento                         99 non-null object
cep                                 129 non-null object
bairro                              187 non-null obje

Cruza em busca de fornecedores

In [98]:
nova = policiais.apply(compara, axis=1) #lê dataframe e chama a função

policiais['cnpj_inteiro'] = nova 

/home/reinaldo/Documentos/Code/intercept/seguranca/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
pagamentos = pd.merge(policiais.drop_duplicates('cnpj_inteiro'), fornecedores_rj, left_on='cnpj_inteiro', right_on='Favorecido')

In [100]:
pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269 entries, 0 to 268
Data columns (total 42 columns):
cnpj                                269 non-null object
CNAE_principal                      269 non-null object
nome_empresa                        269 non-null object
nome_empresa_maiuscula_minuscula    269 non-null object
estado                              269 non-null object
indicador                           269 non-null float64
documento                           269 non-null object
qualificacao                        269 non-null object
socio                               269 non-null object
socio_maiuscula_minuscula           269 non-null object
data_abertura                       269 non-null object
logradouro                          269 non-null object
numero_logradouro                   269 non-null object
complemento                         0 non-null object
cep                                 37 non-null object
bairro                              269 non-null object
m

In [101]:
pagamentos.to_csv('pagamentos_fornecedores_seguranca_rj.csv', index=False, decimal=',')

Rotina para transformar valor em float

In [102]:
def limpeza1(linha):
    valor = linha["Valor_OB"]
    valor = valor.strip()
    valor = valor.replace('.', '')
    valor = valor.replace(',', '.')
    return str(valor)

In [103]:
pagamentos["Valor_OB"] = pagamentos.apply(limpeza1, axis=1)

In [104]:
pagamentos['Valor_OB'] = pagamentos.Valor_OB.astype('float64')

In [105]:
pd.options.display.float_format = '{:,.2f}'.format

Empresas com maiores pagamentos

In [106]:
pagamentos.groupby("Nome Favorecido").Valor_OB.sum().reset_index().sort_values("Valor_OB", ascending=False)

,Nome Favorecido,Valor_OB
1,Persona Vigilancia E Seguranca Ltda,"13,604,790.76"
0,Best Vigilancia E Seguranca Ltda,"4,721,617.43"
2,RIOFORTE VIGILÂNCIA E SEGURANÇA PRIVADA LTDA EPP,"516,782.62"
